# MLflow Training Tutorial

This `train.pynb` Jupyter notebook predicts the quality of wine using [sklearn.linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).  

> This is the Jupyter notebook version of the `train.py` example

Attribution
* The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
* P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
* Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.


In [1]:
!pip install -U mlflow

Requirement already up-to-date: mlflow in /Users/admin/miniconda3/lib/python3.7/site-packages (1.13.1)


In [3]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [4]:
mlflow.set_tracking_uri("http://mlflow.datarevenue.com:5000")

In [5]:
mlflow.create_experiment("mlops-demo")

'1'

In [18]:
# Wine Quality Sample
def train(in_alpha, in_l1_ratio):

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run(experiment_id=1):
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [19]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


In [20]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7336400911821402
  MAE: 0.5643841279275428
  R2: 0.23739466063584158


In [21]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984664
  R2: 0.2799376066653344


In [22]:
!curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.5, 0.29, 48, 0.98, 6.2, 29, 3.33, 1.2, 0, 175, 0.66]]}' http://127.0.0.1:1234/invocations

[20.689054018723663]

In [23]:
exp = mlflow.get_experiment_by_name("mlops-demo")

In [24]:
exp.artifact_location

's3://mlops-model-artifact/1'

In [3]:
!kubectl apply -f seldon-deployment.yaml -n stage

seldondeployment.machinelearning.seldon.io/wines-classifier unchanged


In [66]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=wines-classifier -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "wines-classifier-model-a-0-wines-classifier" rollout to finish: 1 old replicas are pending termination...
Waiting for deployment "wines-classifier-model-a-0-wines-classifier" rollout to finish: 1 old replicas are pending termination...
deployment "wines-classifier-model-a-0-wines-classifier" successfully rolled out


In [2]:
!kubectl delete -f seldon-deployment.yaml

seldondeployment.machinelearning.seldon.io "wines-classifier" deleted


In [30]:
!curl -X POST -H "Content-Type:application/json; format=pandas-split" --data  http://127.0.0.1:9000/predictions

curl: (52) Empty reply from server


In [67]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{\
        "data": { \
            "names": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"], \
            "ndarray": [[7,0.27,0.36,20.7,0.045,45,570,1.001,3,0.45,8.8]] \
        } \
    }' \
    http://a2a2d5f20a12342368355def4eb18716-1626501488.eu-west-1.elb.amazonaws.com:80/seldon/stage/wines-classifier/api/v1.0/predictions

{"data":{"names":[],"ndarray":[3.0861741827087834]},"meta":{"requestPath":{"wines-classifier":"seldonio/mlflowserver:1.6.0"}}}
